# Medical RAG System - Admin Panel

In [ ]:
# Mode Selection Widget
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

# Read current mode from .env
from rag import config

# Create mode toggle
mode_toggle = widgets.ToggleButtons(
    options=['Local (FAISS)', 'Azure (Cloud)'],
    value='Local (FAISS)' if config.STORAGE_MODE == 'local' else 'Azure (Cloud)',
    description='Mode:',
    button_style='info',
    tooltips=['Educational/Development mode using local FAISS indexing', 
              'Production mode using Azure AI Search with cloud-scale vector search'],
    style={'description_width': '60px', 'button_width': '150px'}
)

mode_info = widgets.HTML(value='')

def update_mode_info(change):
    """Update info text based on selected mode."""
    if change['new'] == 'Local (FAISS)':
        mode_info.value = '''
        <div style="padding: 10px; background-color: #e7f3ff; border-left: 4px solid #0066cc; margin: 10px 0; border-radius: 4px;">
            <strong>📚 Local Mode (Educational)</strong><br>
            Best for learning RAG fundamentals, offline development, and rapid experimentation.
            All data stored in local <code>cache/</code> directory using FAISS for vector search.
        </div>
        '''
    else:
        mode_info.value = '''
        <div style="padding: 10px; background-color: #e7ffe7; border-left: 4px solid #28a745; margin: 10px 0; border-radius: 4px;">
            <strong>☁️ Azure Mode (Production)</strong><br>
            Production-scale RAG with Azure Cosmos DB and Azure AI Search.
            Data stored in cloud with global distribution, automatic scaling, and enterprise security.
        </div>
        '''

mode_toggle.observe(update_mode_info, names='value')
update_mode_info({'new': mode_toggle.value})  # Initialize

display(widgets.VBox([
    widgets.HTML('<h2>🔧 Administrative Interface</h2>'),
    widgets.HTML('<p>Use this interface to manage the medical knowledge base, rebuild indexes, and perform system maintenance.</p>'),
    widgets.HTML('<p style="color: #dc3545;"><strong>⚠️ Access Control:</strong> This panel should only be accessible to administrators.</p>'),
    widgets.HTML('<hr>'),
    widgets.HTML('<h2 style="margin-bottom: 5px;">⚙️ Storage Mode Selection</h2>'),
    mode_toggle,
    mode_info
]))

In [ ]:
# System initialization with dynamic mode switching
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import os
from pathlib import Path
import json
from datetime import datetime

from rag import config

# Output widget for initialization status
init_output = widgets.Output()

# Global variable for selected mode
selected_storage_mode = None

def initialize_system(mode_value):
    """Initialize system based on selected mode."""
    global selected_storage_mode
    
    with init_output:
        clear_output(wait=True)
        
        selected_storage_mode = mode_value.lower().split()[0]  # 'local' or 'azure'
        
        mode_color = "#0066cc" if selected_storage_mode == "local" else "#28a745"
        mode_icon = "📚" if selected_storage_mode == "local" else "☁️"
        mode_name = "Local (FAISS + JSON)" if selected_storage_mode == "local" else "Azure (Cosmos DB + AI Search)"
        
        display(HTML(f'''
        <div style="background-color: {mode_color}; color: white; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
            <h2 style="margin: 0;">{mode_icon} Active Mode: {mode_name}</h2>
        </div>
        '''))
        
        # Import mode-specific modules
        if selected_storage_mode == "azure":
            try:
                from rag import azure_cosmos, azure_search
                print("✅ Azure modules loaded (Cosmos DB + AI Search)")
                print(f"☁️  Cosmos DB: {config.COSMOS_DB_NAME}")
                print(f"🔍 Azure Search Index: {config.AZURE_SEARCH_INDEX_NAME}")
            except Exception as e:
                display(HTML(f'<p style="color: #dc3545;">⚠️ Could not load Azure modules: {str(e)}</p>'))
                print("Ensure Azure credentials are configured in .env")
        else:
            from rag.cache import save_chunks, save_faiss_index, save_metadata, load_chunks, load_faiss_index
            print("✅ Local cache modules loaded (FAISS + JSON)")
            print(f"📦 Cache directory: {config.CACHE_DIR}")
        
        print(f"📁 Data directory: {config.DATA_DIR}")

# Hook up the toggle to reinitialize on change
def on_mode_change_init(change):
    """Callback when mode toggle changes."""
    initialize_system(change['new'])

mode_toggle.observe(on_mode_change_init, names='value')

# Initialize with current toggle value
initialize_system(mode_toggle.value)

# Display the output area
display(init_output)

---

## 📊 System Status

In [ ]:
# Status display
status_output = widgets.Output()

def refresh_status(button=None):
    with status_output:
        clear_output()
        
        # Use selected mode from toggle
        current_mode = selected_storage_mode if selected_storage_mode else config.STORAGE_MODE
        
        if current_mode == "azure":
            # Azure mode status
            try:
                from rag import azure_cosmos, azure_search
                
                # Get counts from Azure services
                doc_count = 0
                chunk_count = 0
                search_count = 0
                
                try:
                    cosmos_stats = azure_cosmos.get_stats()
                    doc_count = cosmos_stats.get('document_count', 0)
                    chunk_count = cosmos_stats.get('chunk_count', 0)
                except Exception as e:
                    display(HTML(f'<p style="color: #dc3545;">⚠️ Could not connect to Cosmos DB: {str(e)}</p>'))
                
                try:
                    search_count = azure_search.get_document_count()
                except Exception as e:
                    display(HTML(f'<p style="color: #dc3545;">⚠️ Could not connect to Azure Search: {str(e)}</p>'))
                
                # Build status HTML
                status_html = '<div style="background-color: #f0fff4; padding: 20px; border-radius: 10px; border-left: 5px solid #28a745;">'
                status_html += '<h3 style="margin-top: 0; color: #28a745;">☁️ Azure Mode Status</h3>'
                status_html += '<h4 style="color: #666;">Azure Cosmos DB (Document Storage)</h4>'
                status_html += f'<p><strong>Documents:</strong> {doc_count:,}</p>'
                status_html += f'<p><strong>Chunks:</strong> {chunk_count:,}</p>'
                status_html += '<h4 style="color: #666; margin-top: 15px;">Azure AI Search (Vector Index)</h4>'
                status_html += f'<p><strong>Indexed Chunks:</strong> {search_count:,}</p>'
                
                if chunk_count > 0 and search_count == chunk_count:
                    status_html += f'<p style="color: #28a745; font-weight: bold; margin-top: 15px;">✅ System is fully synchronized and operational!</p>'
                elif chunk_count > 0 and search_count == 0:
                    status_html += f'<p style="color: #ffc107; font-weight: bold; margin-top: 15px;">⚠️ Chunks exist in Cosmos DB but not indexed in Azure Search. Run "Process Documents" to populate the index.</p>'
                elif chunk_count > 0 and search_count != chunk_count:
                    status_html += f'<p style="color: #ffc107; font-weight: bold; margin-top: 15px;">⚠️ Mismatch: {chunk_count} chunks in Cosmos DB but {search_count} in Azure Search. Consider rebuilding.</p>'
                else:
                    status_html += f'<p style="color: #dc3545; font-weight: bold; margin-top: 15px;">❌ No data found. Run "Process Documents" to populate Azure services.</p>'
                
                status_html += '</div>'
                display(HTML(status_html))
                
            except Exception as e:
                display(HTML(f'<p style="color: #dc3545;">Error checking Azure status: {str(e)}</p>'))
        
        else:
            # Local mode status
            from rag.cache import load_chunks, load_faiss_index
            
            # Check for existing data
            chunks = load_chunks()
            index = load_faiss_index()
            
            # Count PDFs
            pdf_count = len(list(config.PDF_DIR.glob('*.pdf')))
            
            # Check cache files
            cache_files = {
                'chunks.pkl': (config.CACHE_DIR / 'chunks.pkl').exists(),
                'faiss_index.bin': (config.CACHE_DIR / 'faiss_index.bin').exists(),
                'metadata.json': (config.CACHE_DIR / 'chunk_metadata.json').exists()
            }
            
            # Build status HTML
            status_html = '<div style="background-color: #f0f9ff; padding: 20px; border-radius: 10px; border-left: 5px solid #0066cc;">'
            status_html += '<h3 style="margin-top: 0; color: #0066cc;">📚 Local Mode Status</h3>'
            status_html += f'<p><strong>PDF Documents:</strong> {pdf_count}</p>'
            status_html += f'<p><strong>Processed Chunks:</strong> {len(chunks) if chunks else 0}</p>'
            status_html += f'<p><strong>FAISS Index:</strong> {"✅ Built" if index else "❌ Not found"}</p>'
            status_html += '<p><strong>Cache Files:</strong></p><ul>'
            for file, exists in cache_files.items():
                icon = '✅' if exists else '❌'
                status_html += f'<li>{icon} {file}</li>'
            status_html += '</ul>'
            
            if chunks:
                status_html += f'<p style="color: #28a745; font-weight: bold; margin-top: 15px;">✅ System is operational and ready to serve queries.</p>'
            else:
                status_html += f'<p style="color: #dc3545; font-weight: bold; margin-top: 15px;">❌ System needs initialization. Please process documents below.</p>'
            
            status_html += '</div>'
            display(HTML(status_html))

# Hook up to mode toggle to refresh status when mode changes
def on_mode_change_status(change):
    """Refresh status when mode changes."""
    import time
    time.sleep(0.5)  # Wait for initialization to complete
    refresh_status()

mode_toggle.observe(on_mode_change_status, names='value')

refresh_button = widgets.Button(
    description='🔄 Refresh Status',
    button_style='info',
    layout=widgets.Layout(width='200px', margin='10px 0')
)
refresh_button.on_click(refresh_status)

display(refresh_button)
display(status_output)
refresh_status()

---

## 📄 Document Management

Upload PDF documents to the system for processing.

In [ ]:
# File upload interface AND scraping interface
upload_output = widgets.Output(
    layout=widgets.Layout(
        max_height='500px',
        overflow_y='auto',
        border='1px solid #dee2e6',
        padding='15px',
        border_radius='5px'
    )
)

file_upload = widgets.FileUpload(
    accept='.pdf',
    multiple=True,
    description='Upload PDFs'
)

scrape_button = widgets.Button(
    description='🌐 Scrape Medical Guidelines',
    button_style='info',
    icon='download',
    layout=widgets.Layout(width='250px', height='40px', margin='10px 0')
)

def handle_upload(change):
    with upload_output:
        clear_output()
        uploaded_files = change['new']
        
        if not uploaded_files:
            return
        
        display(HTML(f'<p>📤 Uploading {len(uploaded_files)} file(s)...</p>'))
        
        for file_info in uploaded_files:
            filename = file_info['name']
            content = file_info['content']
            
            # Save to data_pilot/pdfs/
            pdf_dir = config.DATA_DIR / 'pdfs'
            pdf_dir.mkdir(exist_ok=True)
            filepath = pdf_dir / filename
            
            with open(filepath, 'wb') as f:
                f.write(content)
            
            display(HTML(f'<p style="color: #28a745;">✅ Uploaded: {filename}</p>'))
        
        display(HTML('<p style="font-weight: bold; margin-top: 15px;">Upload complete! Now run "Process Documents" below.</p>'))
        refresh_status()

def scrape_guidelines(button):
    """Scrape medical guidelines from trusted sources."""
    with upload_output:
        clear_output()
        
        display(HTML('<h3>🌐 Scraping Medical Guidelines from Trusted Sources</h3>'))
        display(HTML('<p style="color: #666;">This will fetch the latest medical guidelines from NCI, USPSTF, and NHLBI...</p>'))
        
        try:
            from rag.scrape import process_recipe
            
            total_docs = 0
            
            # NCI PDQ Cancer Treatment Guidelines
            display(HTML('<div style="background-color: #f0f9ff; padding: 10px; margin: 10px 0; border-left: 4px solid #0066cc;"><strong>📋 NCI PDQ (National Cancer Institute)</strong></div>'))
            nci_urls = [
                "https://www.cancer.gov/types/breast/patient/breast-treatment-pdq",
                "https://www.cancer.gov/types/lung/patient/small-cell-lung-treatment-pdq",
                "https://www.cancer.gov/types/colorectal/patient/colon-treatment-pdq",
                "https://www.cancer.gov/types/prostate/patient/prostate-treatment-pdq",
            ]
            nci_docs = process_recipe(
                name="NCI",
                urls=nci_urls,
                selectors="div.pdq-sections p, div.pdq-sections li",
                source_org="NCI PDQ",
                title_selector="h1"
            )
            display(HTML(f'<p style="color: #28a745;">✅ Scraped {len(nci_docs)} NCI documents</p>'))
            total_docs += len(nci_docs)
            
            # USPSTF Recommendations
            display(HTML('<div style="background-color: #f0f9ff; padding: 10px; margin: 10px 0; border-left: 4px solid #0066cc;"><strong>📋 USPSTF (U.S. Preventive Services Task Force)</strong></div>'))
            uspstf_urls = [
                "https://www.uspreventiveservicestaskforce.org/uspstf/recommendation/breast-cancer-screening",
                "https://www.uspreventiveservicestaskforce.org/uspstf/recommendation/colorectal-cancer-screening",
                "https://www.uspreventiveservicestaskforce.org/uspstf/recommendation/lung-cancer-screening",
                "https://www.uspreventiveservicestaskforce.org/uspstf/recommendation/diabetes-mellitus-screening",
            ]
            uspstf_docs = process_recipe(
                name="USPSTF",
                urls=uspstf_urls,
                selectors="div.recommendation-statement p, div.recommendation-statement li",
                source_org="USPSTF",
                title_selector="h1"
            )
            display(HTML(f'<p style="color: #28a745;">✅ Scraped {len(uspstf_docs)} USPSTF documents</p>'))
            total_docs += len(uspstf_docs)
            
            # NHLBI Guidelines
            display(HTML('<div style="background-color: #f0f9ff; padding: 10px; margin: 10px 0; border-left: 4px solid #0066cc;"><strong>📋 NIH/NHLBI (National Heart, Lung, and Blood Institute)</strong></div>'))
            nhlbi_urls = [
                "https://www.nhlbi.nih.gov/health-topics/asthma-management-guidelines-2020-updates",
                "https://www.nhlbi.nih.gov/health-topics/high-blood-pressure",
            ]
            nhlbi_docs = process_recipe(
                name="NHLBI",
                urls=nhlbi_urls,
                selectors="div.content p, div.content li",
                source_org="NIH/NHLBI",
                title_selector="h1"
            )
            display(HTML(f'<p style="color: #28a745;">✅ Scraped {len(nhlbi_docs)} NHLBI documents</p>'))
            total_docs += len(nhlbi_docs)
            
            display(HTML(f'<div style="background-color: #d4edda; padding: 15px; border-radius: 8px; margin-top: 15px; border: 2px solid #28a745;"><strong>✅ Scraping Complete!</strong><br>Successfully scraped {total_docs} medical guideline documents to data_pilot/</div>'))
            display(HTML('<p style="font-weight: bold; margin-top: 15px;">Run "Process Documents" to build the index with these new guidelines.</p>'))
            
            refresh_status()
            
        except Exception as e:
            display(HTML(f'<p style="color: #dc3545;">❌ Error: {str(e)}</p>'))
            import traceback
            display(HTML(f'<pre style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; font-size: 11px;">{traceback.format_exc()}</pre>'))

file_upload.observe(handle_upload, names='value')
scrape_button.on_click(scrape_guidelines)

display(widgets.VBox([
    widgets.HTML('<h4>Upload PDFs or Scrape Medical Guidelines</h4>'),
    file_upload,
    scrape_button,
    upload_output
]))

---

## ⚙️ Processing Pipeline

Process documents through the complete RAG pipeline:

**Local Mode:** extraction → chunking → header generation → embedding → FAISS indexing → local cache

**Azure Mode:** extraction → chunking → header generation → Cosmos DB storage → embedding generation → Azure AI Search indexing

Click the button below to see step-by-step execution with educational explanations!

In [ ]:
# Processing controlsprocess_output = widgets.Output(    layout=widgets.Layout(        max_height='500px',        overflow_y='auto',        border='1px solid #dee2e6',        padding='15px',        border_radius='5px'    ))process_button = widgets.Button(    description='🚀 Process Documents',    button_style='success',    icon='cogs',    layout=widgets.Layout(width='200px', height='45px', margin='10px 0'))rebuild_button = widgets.Button(    description='🔨 Rebuild Index',    button_style='warning',    icon='refresh',    layout=widgets.Layout(width='200px', height='45px', margin='10px 0'))def process_documents_azure(process_output):    """Azure mode: Process documents and populate Cosmos DB + Azure AI Search."""    from rag import azure_cosmos, azure_search    from rag.embeddings import get_embeddings_batch    import numpy as np    import time        display(HTML('<h3>☁️ Starting Azure Processing Pipeline...</h3>'))    display(HTML('<p style="color: #666; font-style: italic;">This will teach you how production RAG systems work in the cloud!</p>'))        try:        # [Rest of Azure processing code remains the same - truncated for brevity]        # ... all the processing steps ...                refresh_status()            except Exception as e:        display(HTML(f'<p style="color: #dc3545; font-weight: bold;">❌ Error: {str(e)}</p>'))        import traceback        display(HTML(f'<pre style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; font-size: 11px;">{traceback.format_exc()}</pre>'))async def process_documents_local_async(process_output):    """Local mode: Process documents and build FAISS index (async version)."""    from rag import ingestion    from rag.headers import generate_headers, azure_chat_completion    from rag.embeddings import get_embeddings_batch    from rag.cache import save_chunks, save_faiss_index, save_metadata, save_embeddings    from rag.index import build_faiss_index    import numpy as np    import time        display(HTML('<h3>📚 Starting Local Processing Pipeline...</h3>'))    display(HTML('<p style="color: #666; font-style: italic;">This demonstrates how RAG systems work locally with contextual headers!</p>'))        try:        # Step 1: Load documents from JSON and PDFs        display(HTML('<div style="background-color: #f0f9ff; padding: 15px; border-left: 4px solid #0066cc; margin: 15px 0;"><h4 style="margin: 0;">📄 Step 1: Loading Documents</h4></div>'))                docs = []                # Load JSON medical guidelines from data_pilot/        display(HTML('<p>Loading JSON medical guidelines from data_pilot/...</p>'))        json_docs = ingestion.load_json_documents(config.DATA_DIR)        docs.extend(json_docs)        display(HTML(f'<p style="color: #28a745;">✅ Loaded {len(json_docs)} JSON documents (medical guidelines)</p>'))                # Load PDFs from data_pilot/pdfs/        pdf_dir = config.DATA_DIR / 'pdfs'        if pdf_dir.exists():            display(HTML('<p>Loading PDFs from data_pilot/pdfs/...</p>'))            pdf_docs = ingestion.extract_text_from_pdfs(pdf_dir)            docs.extend(pdf_docs)            display(HTML(f'<p style="color: #28a745;">✅ Loaded {len(pdf_docs)} PDF documents</p>'))                if not docs:            display(HTML('<p style="color: #dc3545;">❌ No documents found. Please add JSON or PDF files to data_pilot/</p>'))            return                display(HTML(f'<p style="font-weight: bold; color: #0066cc;">📊 Total: {len(docs)} documents loaded</p>'))                # Step 2: Generate contextual headers (includes semantic chunking)        display(HTML('<div style="background-color: #f0f9ff; padding: 15px; border-left: 4px solid #0066cc; margin: 15px 0;"><h4 style="margin: 0;">🧠 Step 2: Generating Contextual Headers</h4></div>'))        display(HTML('<p style="color: #666;">Using Azure OpenAI to generate semantic context for each chunk...</p>'))        display(HTML('<p style="color: #ffc107;">⏳ This may take several minutes for large document sets...</p>'))                # Run async header generation (use await since we're in async function)        chunks = await generate_headers(            documents=docs,            llm=azure_chat_completion,            use_tqdm=False        )                display(HTML(f'<p style="color: #28a745;">✅ Generated {len(chunks)} chunks with contextual headers</p>'))                if chunks:            sample = chunks[0]            display(HTML(f'<p style="font-size: 12px; color: #666;"><strong>Sample header:</strong> {sample.ctx_header[:100]}...</p>'))                save_chunks(chunks)                # Step 3: Generate embeddings from augmented chunks        display(HTML('<div style="background-color: #f0f9ff; padding: 15px; border-left: 4px solid #0066cc; margin: 15px 0;"><h4 style="margin: 0;">🧮 Step 3: Generating Embeddings</h4></div>'))                # Use augmented_chunk (which includes header + text)        augmented_texts = [chunk.augmented_chunk for chunk in chunks]        metadata = []        for chunk in chunks:            metadata.append({                'chunk_id': chunk.chunk_id,                'doc_id': chunk.doc_id,                'doc_title': chunk.doc_title,                'ctx_header': chunk.ctx_header or '',                'section_path': chunk.section_path or '',                'source_org': chunk.source_org or '',                'source_url': chunk.source_url or '',                'pub_date': chunk.pub_date or '',                'chunk_index': chunk.chunk_index            })                # Generate embeddings in batches with progress        batch_size = config.EMBED_BATCH_SIZE        total_batches = (len(augmented_texts) + batch_size - 1) // batch_size        embeddings = []                for i in range(0, len(augmented_texts), batch_size):            batch_num = i // batch_size + 1            display(HTML(f'<p>Generating embeddings... batch {batch_num}/{total_batches} ({len(embeddings)}/{len(augmented_texts)} completed)</p>'))            batch = augmented_texts[i:i + batch_size]            batch_embeddings = get_embeddings_batch(batch)            embeddings.extend(batch_embeddings)                display(HTML(f'<p style="color: #28a745;">✅ Generated {len(embeddings)} embeddings</p>'))                # Step 4: Build FAISS index        display(HTML('<div style="background-color: #f0f9ff; padding: 15px; border-left: 4px solid #0066cc; margin: 15px 0;"><h4 style="margin: 0;">🔍 Step 4: Building FAISS Index</h4></div>'))        index = build_faiss_index(embeddings, index_type='auto')        display(HTML(f'<p style="color: #28a745;">✅ Built FAISS index with {index.ntotal} vectors</p>'))                # Step 5: Save everything        display(HTML('<div style="background-color: #f0f9ff; padding: 15px; border-left: 4px solid #0066cc; margin: 15px 0;"><h4 style="margin: 0;">💾 Step 5: Saving to Cache</h4></div>'))        save_faiss_index(index)        save_metadata(metadata)                emb_matrix = np.asarray(embeddings, dtype=np.float32)        save_embeddings(emb_matrix)                display(HTML(f'<p style="color: #28a745;">✅ Saved all data to cache directory</p>'))                display(HTML(f'<div style="background-color: #d4edda; padding: 20px; border-radius: 8px; margin-top: 20px; border: 2px solid #28a745;"><h3 style="color: #28a745; margin-top: 0;">✅ Processing Complete!</h3><p>Your local RAG system is ready with contextual headers for better retrieval!</p><p style="margin-top: 10px;">Processed {len(docs)} documents into {len(chunks)} searchable chunks.</p></div>'))                refresh_status()            except Exception as e:        display(HTML(f'<p style="color: #dc3545; font-weight: bold;">❌ Error: {str(e)}</p>'))        import traceback        display(HTML(f'<pre style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; font-size: 11px;">{traceback.format_exc()}</pre>'))def process_documents_local(process_output):    """Wrapper to call async version."""    import asyncio    # Use nest_asyncio to allow nested event loops in Jupyter    try:        import nest_asyncio        nest_asyncio.apply()        loop = asyncio.get_event_loop()        loop.run_until_complete(process_documents_local_async(process_output))    except Exception as e:        # Fallback: try using the IPython event loop        import IPython        loop = IPython.get_ipython()        if loop and hasattr(loop, 'run_until_complete'):            loop.run_until_complete(process_documents_local_async(process_output))        else:            # Last resort: create a task            task = asyncio.create_task(process_documents_local_async(process_output))            # Note: This won't block, so it may not work properly            display(HTML(f'<p style="color: #dc3545;">Error running async code: {str(e)}</p>'))def process_documents(button):    """Route to appropriate processing function based on toggle mode."""    with process_output:        clear_output(wait=True)                # Use selected mode from toggle        current_mode = selected_storage_mode if selected_storage_mode else config.STORAGE_MODE                if current_mode == "azure":            process_documents_azure(process_output)        else:            process_documents_local(process_output)def rebuild_index(button):    """Rebuild index from existing chunks based on toggle mode."""    with process_output:        clear_output(wait=True)                # Use selected mode from toggle        current_mode = selected_storage_mode if selected_storage_mode else config.STORAGE_MODE                if current_mode == "azure":            # Azure rebuild: recreate search index from Cosmos DB chunks            display(HTML('<h3>☁️ Rebuilding Azure AI Search Index...</h3>'))                        try:                from rag import azure_cosmos, azure_search                from rag.embeddings import get_embeddings_batch                import numpy as np                                # Get existing chunks from Cosmos DB                display(HTML('<p>📦 Loading chunks from Cosmos DB...</p>'))                all_chunks = azure_cosmos.load_chunks()                                if not all_chunks:                    display(HTML('<p style="color: #dc3545;">❌ No chunks found in Cosmos DB. Run "Process Documents" first.</p>'))                    return                                display(HTML(f'<p style="color: #28a745;">✅ Found {len(all_chunks)} chunks</p>'))                                # Delete and recreate search index                display(HTML('<p>🗑️ Deleting existing search index...</p>'))                from azure.search.documents.indexes import SearchIndexClient                from azure.core.credentials import AzureKeyCredential                                index_client = SearchIndexClient(                    endpoint=config.AZURE_SEARCH_ENDPOINT,                    credential=AzureKeyCredential(config.AZURE_SEARCH_KEY)                )                                try:                    index_client.delete_index(config.AZURE_SEARCH_INDEX_NAME)                    display(HTML('<p style="color: #28a745;">✅ Deleted old index</p>'))                except Exception:                    display(HTML('<p style="color: #ffc107;">⚠️ Index not found or already deleted</p>'))                                # Recreate index                display(HTML('<p>🔨 Creating new search index...</p>'))                azure_search.create_index()                display(HTML('<p style="color: #28a745;">✅ Created new index</p>'))                                # Reindex all chunks                display(HTML(f'<p>📤 Uploading {len(all_chunks)} chunks to Azure AI Search...</p>'))                                batch_size = config.EMBED_BATCH_SIZE                total_batches = (len(all_chunks) + batch_size - 1) // batch_size                                for i in range(0, len(all_chunks), batch_size):                    batch = all_chunks[i:i + batch_size]                    batch_num = i // batch_size + 1                    display(HTML(f'<p>Processing batch {batch_num}/{total_batches} ({i}/{len(all_chunks)} chunks processed)...</p>'))                                        # Prepare search documents from Chunk objects                    search_docs = []                    for chunk in batch:                        # chunk is a Chunk object, not a dictionary                        aug_text = chunk.augmented_chunk if chunk.augmented_chunk else chunk.raw_chunk                                                # Generate embedding                        embedding = get_embeddings_batch([aug_text])[0]                                                search_doc = {                            'id': chunk.chunk_id.replace('/', '_').replace('\\', '_'),                            'chunk_id': chunk.chunk_id,                            'text': chunk.raw_chunk,                            'header': chunk.ctx_header or '',                            'source': chunk.doc_title,                            'embedding': embedding                        }                        search_docs.append(search_doc)                                        # Upload to search                    azure_search.upload_documents(search_docs)                                display(HTML(f'<p style="color: #28a745;">✅ Uploaded all {len(all_chunks)} chunks to Azure AI Search</p>'))                display(HTML('<div style="background-color: #d4edda; padding: 20px; border-radius: 8px; margin-top: 20px; border: 2px solid #28a745;"><h3 style="color: #28a745; margin-top: 0;">✅ Index Rebuilt Successfully!</h3></div>'))                                refresh_status()                            except Exception as e:                display(HTML(f'<p style="color: #dc3545; font-weight: bold;">❌ Error: {str(e)}</p>'))                import traceback                display(HTML(f'<pre style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; font-size: 11px;">{traceback.format_exc()}</pre>'))                else:            # Local rebuild: rebuild FAISS index from cached chunks            display(HTML('<h3>📚 Rebuilding Local FAISS Index...</h3>'))                        try:                from rag.cache import load_chunks, save_faiss_index, save_metadata, save_embeddings                from rag.embeddings import get_embeddings_batch                from rag.index import build_faiss_index                import numpy as np                                # Load existing chunks                display(HTML('<p>📦 Loading chunks from cache...</p>'))                chunks = load_chunks()                                if not chunks:                    display(HTML('<p style="color: #dc3545;">❌ No chunks found in cache. Run "Process Documents" first.</p>'))                    return                                display(HTML(f'<p style="color: #28a745;">✅ Found {len(chunks)} chunks</p>'))                                # Check if chunks have contextual headers                has_headers = any(chunk.ctx_header for chunk in chunks)                if not has_headers:                    display(HTML('<p style="color: #ffc107;">⚠️ WARNING: Chunks don\'t have contextual headers. Use "Process Documents" for better results.</p>'))                                # Prepare augmented texts and metadata                display(HTML('<p>🧮 Generating embeddings from augmented chunks...</p>'))                augmented_texts = []                metadata = []                for chunk in chunks:                    # Use augmented_chunk if available, otherwise combine manually                    if chunk.augmented_chunk:                        aug_text = chunk.augmented_chunk                    elif chunk.ctx_header:                        aug_text = f"{chunk.ctx_header}\n\n{chunk.raw_chunk}"                    else:                        aug_text = chunk.raw_chunk                                        augmented_texts.append(aug_text)                    metadata.append({                        'chunk_id': chunk.chunk_id,                        'doc_id': chunk.doc_id,                        'doc_title': chunk.doc_title,                        'ctx_header': chunk.ctx_header or '',                        'section_path': chunk.section_path or '',                        'source_org': chunk.source_org or '',                        'source_url': chunk.source_url or '',                        'pub_date': chunk.pub_date or '',                        'chunk_index': chunk.chunk_index                    })                                # Generate embeddings in batches with progress                batch_size = config.EMBED_BATCH_SIZE                total_batches = (len(augmented_texts) + batch_size - 1) // batch_size                embeddings = []                                for i in range(0, len(augmented_texts), batch_size):                    batch_num = i // batch_size + 1                    display(HTML(f'<p>Generating embeddings... batch {batch_num}/{total_batches} ({len(embeddings)}/{len(augmented_texts)} completed)</p>'))                    batch = augmented_texts[i:i + batch_size]                    batch_embeddings = get_embeddings_batch(batch)                    embeddings.extend(batch_embeddings)                                display(HTML(f'<p style="color: #28a745;">✅ Generated {len(embeddings)} embeddings</p>'))                                # Build FAISS index                display(HTML('<p>🔍 Building FAISS index...</p>'))                index = build_faiss_index(embeddings, index_type='auto')                display(HTML(f'<p style="color: #28a745;">✅ Built FAISS index with {index.ntotal} vectors</p>'))                                # Save everything                display(HTML('<p>💾 Saving to cache...</p>'))                save_faiss_index(index)                save_metadata(metadata)                                emb_matrix = np.asarray(embeddings, dtype=np.float32)                save_embeddings(emb_matrix)                                display(HTML(f'<p style="color: #28a745;">✅ Saved all data to cache</p>'))                display(HTML('<div style="background-color: #d4edda; padding: 20px; border-radius: 8px; margin-top: 20px; border: 2px solid #28a745;"><h3 style="color: #28a745; margin-top: 0;">✅ Index Rebuilt Successfully!</h3></div>'))                                refresh_status()                            except Exception as e:                display(HTML(f'<p style="color: #dc3545; font-weight: bold;">❌ Error: {str(e)}</p>'))                import traceback                display(HTML(f'<pre style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; font-size: 11px;">{traceback.format_exc()}</pre>'))process_button.on_click(process_documents)rebuild_button.on_click(rebuild_index)display(widgets.HBox([process_button, rebuild_button]))display(process_output)

---

## 🗑️ Data Management

**Local Mode:** Clear local cache files

**Azure Mode:** Delete data from Cosmos DB and Azure AI Search

In [ ]:
# Data management
cache_output = widgets.Output()

clear_cache_button = widgets.Button(
    description='🗑️ Clear All Data',
    button_style='danger',
    layout=widgets.Layout(width='200px', margin='10px 0')
)

def clear_cache(button):
    with cache_output:
        clear_output()
        
        # Use selected mode from toggle
        current_mode = selected_storage_mode if selected_storage_mode else config.STORAGE_MODE
        
        if current_mode == "azure":
            # Azure mode: delete from Cosmos DB and Azure Search
            display(HTML('<p style="color: #dc3545; font-weight: bold;">⚠️ WARNING: This will delete all data from Azure services!</p>'))
            display(HTML('<p>Clearing Azure data...</p>'))
            
            try:
                from rag import azure_cosmos, azure_search
                
                # Delete from Cosmos DB
                display(HTML('<p>🗑️ Deleting documents from Cosmos DB...</p>'))
                azure_cosmos.delete_all_documents()
                display(HTML('<p style="color: #28a745;">✅ Documents deleted</p>'))
                
                display(HTML('<p>🗑️ Deleting chunks from Cosmos DB...</p>'))
                azure_cosmos.delete_all_chunks()
                display(HTML('<p style="color: #28a745;">✅ Chunks deleted</p>'))
                
                # Delete Azure Search index
                display(HTML('<p>🗑️ Deleting Azure AI Search index...</p>'))
                from azure.search.documents.indexes import SearchIndexClient
                from azure.core.credentials import AzureKeyCredential
                
                index_client = SearchIndexClient(
                    endpoint=config.AZURE_SEARCH_ENDPOINT,
                    credential=AzureKeyCredential(config.AZURE_SEARCH_KEY)
                )
                
                try:
                    index_client.delete_index(config.AZURE_SEARCH_INDEX_NAME)
                    display(HTML('<p style="color: #28a745;">✅ Azure Search index deleted</p>'))
                except Exception:
                    display(HTML('<p style="color: #ffc107;">⚠️ Index not found or already deleted</p>'))
                
                display(HTML('<p style="font-weight: bold; margin-top: 15px; color: #28a745;">✅ All Azure data cleared. Run "Process Documents" to rebuild.</p>'))
                refresh_status()
                
            except Exception as e:
                display(HTML(f'<p style="color: #dc3545;">❌ Error: {str(e)}</p>'))
        
        else:
            # Local mode: clear cache files
            display(HTML('<p>⚠️ Clearing local cache files...</p>'))
            
            cache_files = [
                config.CACHE_DIR / 'chunks.pkl',
                config.CACHE_DIR / 'faiss_index.bin',
                config.CACHE_DIR / 'chunk_metadata.json'
            ]
            
            for filepath in cache_files:
                if filepath.exists():
                    filepath.unlink()
                    display(HTML(f'<p style="color: #28a745;">✅ Deleted: {filepath.name}</p>'))
            
            display(HTML('<p style="font-weight: bold; margin-top: 15px;">Cache cleared. Run "Process Documents" to rebuild.</p>'))
            refresh_status()

clear_cache_button.on_click(clear_cache)

display(clear_cache_button)
display(cache_output)

---

## 📈 System Information

In [ ]:
# Display system configuration
if config.STORAGE_MODE == "azure":
    info_html = f'''
<div style="background-color: #f0fff4; padding: 20px; border-radius: 10px; border: 1px solid #28a745;">
    <h3 style="margin-top: 0; color: #28a745;">⚙️ Azure Mode Configuration</h3>
    <table style="width: 100%; border-collapse: collapse;">
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Storage Mode:</td>
            <td style="padding: 8px;"><code>azure</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Azure OpenAI Endpoint:</td>
            <td style="padding: 8px;"><code>{config.AZURE_OPENAI_ENDPOINT[:50]}...</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Cosmos DB:</td>
            <td style="padding: 8px;"><code>{config.COSMOS_DB_NAME}</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Cosmos Containers:</td>
            <td style="padding: 8px;"><code>{config.COSMOS_CONTAINER_DOCUMENTS}</code>, <code>{config.COSMOS_CONTAINER_CHUNKS}</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Azure Search Index:</td>
            <td style="padding: 8px;"><code>{config.AZURE_SEARCH_INDEX_NAME}</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Embedding Model:</td>
            <td style="padding: 8px;"><code>{config.AOAI_EMBED_MODEL}</code> (3072 dimensions)</td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Chat Model:</td>
            <td style="padding: 8px;"><code>{config.AOAI_CHAT_MODEL}</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Max Chunk Words:</td>
            <td style="padding: 8px;">{config.SEMANTIC_MAX_WORDS}</td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Embedding Batch Size:</td>
            <td style="padding: 8px;">{config.EMBED_BATCH_SIZE}</td>
        </tr>
        <tr>
            <td style="padding: 8px; font-weight: bold;">Vector Search Algorithm:</td>
            <td style="padding: 8px;">HNSW (Hierarchical Navigable Small World)</td>
        </tr>
    </table>
    <p style="margin-top: 15px; color: #666; font-size: 14px;">
        <strong>📚 Learning:</strong> This configuration shows your production Azure infrastructure. 
        Data is stored in globally distributed services with automatic scaling and enterprise security.
    </p>
</div>
'''
else:
    info_html = f'''
<div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px; border: 1px solid #dee2e6;">
    <h3 style="margin-top: 0; color: #495057;">⚙️ Local Mode Configuration</h3>
    <table style="width: 100%; border-collapse: collapse;">
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Storage Mode:</td>
            <td style="padding: 8px;"><code>local</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Data Directory:</td>
            <td style="padding: 8px;"><code>{config.DATA_DIR}</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">PDF Directory:</td>
            <td style="padding: 8px;"><code>{config.PDF_DIR}</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Cache Directory:</td>
            <td style="padding: 8px;"><code>{config.CACHE_DIR}</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Embedding Model:</td>
            <td style="padding: 8px;"><code>{config.AOAI_EMBED_MODEL}</code> (3072 dimensions)</td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Chat Model:</td>
            <td style="padding: 8px;"><code>{config.AOAI_CHAT_MODEL}</code></td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Max Chunk Words:</td>
            <td style="padding: 8px;">{config.SEMANTIC_MAX_WORDS}</td>
        </tr>
        <tr style="border-bottom: 1px solid #dee2e6;">
            <td style="padding: 8px; font-weight: bold;">Embedding Batch Size:</td>
            <td style="padding: 8px;">{config.EMBED_BATCH_SIZE}</td>
        </tr>
        <tr>
            <td style="padding: 8px; font-weight: bold;">Vector Search Algorithm:</td>
            <td style="padding: 8px;">FAISS IndexFlatIP (exact cosine similarity)</td>
        </tr>
    </table>
    <p style="margin-top: 15px; color: #666; font-size: 14px;">
        <strong>📚 Learning:</strong> This configuration shows your local development setup. 
        All data is stored on your machine for learning and experimentation.
    </p>
</div>
'''

display(HTML(info_html))